In [1]:
import re
import json
import random
import string 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
with open("./Dataset/Intent2.json") as f:
    df = json.load(f)

In [4]:
def text_preprocessing(text):
    text = text.lower()
    # remove punctuation
    text = [letter for letter in text if letter not in string.punctuation]
    
    # join the list of characters into a string
    text = ''.join(text)
    
    # lemmatize
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    
    text = ' '.join(text)
    return text

In [5]:
# get the intent of the message
words = []
documents = []
classes = []
intent_dict = {}

for intent in df['intents']:
    
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
        
    if intent['intent'] not in intent_dict:
        intent_dict[intent['intent']] = []
        
    for text in intent['text']:
        text = text_preprocessing(text)
        words.extend(text.split())
        documents.append((text,intent['intent']))
        
    for response in intent['responses']:
        intent_dict[intent['intent']].append(response)

In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes, file=open('classes.pkl','wb'))
pickle.dump(intent_dict, file=open('intent_dict.pkl','wb'))

In [7]:
training = []
output_empty = [0] * len(classes)

In [8]:
for document in documents:
    bag = []
    text = document[0]
    for w in words:
        if w in text:
            bag.append(1)
        else:
            bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [9]:
random.shuffle(training)
training = np.array(training)

C:\Users\maili\AppData\Local\Temp/ipykernel_10768/2195589081.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [10]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [11]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

d:\Project\Python\Telegram Bot 2\Semi Intelligent Bot\venv\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [12]:
hist = model.fit(np.array(train_x),np.array(train_y),batch_size=5,epochs=200,verbose=1)

Epoch 1/200
24/24 [==============================] - 1s 2ms/step - loss: 2.1747 - accuracy: 0.2083
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 1.8834 - accuracy: 0.3750
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 1.5180 - accuracy: 0.4917
Epoch 4/200
24/24 [==============================] - 0s 2ms/step - loss: 1.3508 - accuracy: 0.6000
Epoch 5/200
24/24 [==============================] - 0s 2ms/step - loss: 1.2967 - accuracy: 0.5667
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 1.1265 - accuracy: 0.6417
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 1.0481 - accuracy: 0.6583
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 0.9630 - accuracy: 0.6833
Epoch 9/200
24/24 [==============================] - 0s 2ms/step - loss: 0.8409 - accuracy: 0.7333
Epoch 10/200
24/24 [==============================] - 0s 2ms/step - loss: 0.7763 - accuracy: 0.7250
Epoch 11/

In [13]:
model.save('chatbot_model.h5',hist)